# Loading Libraries

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
#movies_data=pd.read_csv(r"/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
#credits_data=pd.read_csv(r"/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")
movies_data=pd.read_csv(r"C:\Users\jitan\Referances\content_based movie recommendation system\dataset/tmdb_5000_movies.csv")
credits_data=pd.read_csv(r"C:\Users\jitan\Referances\content_based movie recommendation system\dataset/tmdb_5000_credits.csv")


In [16]:
movies_data.head()
movies_data.columns 
# 'budget','genres','homepage','id','keywords','original_language','original_title','overview','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','tagline','title','vote_average','vote_count'
movies_data.shape  # 4803,20
print(movies_data.isna().sum())


budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64


In [17]:
credits_data.head()
credits_data.columns # 'movie_id', 'title', 'cast', 'crew'
credits_data.shape  # 4803,4
print(credits_data.isna().sum())

movie_id    0
title       0
cast        0
crew        0
dtype: int64


As these data-set ve many columns that are not nessential, so we take columns that are more important

In [18]:
selected_columns=['genres','keywords','overview','production_companies','title']
movies=movies_data[selected_columns]


# Merging 2 data-sets: 

In [19]:
movies=movies.merge(credits_data,on="title")    # As "title" column present on both data-set, so mergeing on"title"


## Data-set insights

In [20]:
movies.head()
movies.shape   # 4809,8

# handling Null values
movies.isna().sum()
movies.fillna("unknown",inplace=True)

# Duplicate values
movies.duplicated().sum()

,genres,keywords,overview,production_companies,title,movie_id,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [21]:
movies.columns

Index(['genres', 'keywords', 'overview', 'production_companies', 'title',
       'movie_id', 'cast', 'crew'],
      dtype='object')

## Preprocessing Data(Column-wise)

In [9]:
#'movie_id' column
movies['movie_id'].head()
movies['movie_id'][0]  

# we no need to preprocess this column

19995

In [10]:
#'title' column
movies['title'].head()
movies['title'][0]  

# we no need to preprocess this column



'Avatar'

In [11]:
## 'genres' column
movies['genres'].head()
movies['genres'][0]

# to make all words lower-case
movies["genres"]=movies["genres"].str.lower()

# Here we only need "names" from "genres"
# we also need to remove " " As 'Science','Fiction' & 'Science Fiction' can be represent as different 
def genre_extractor(df):
    genres=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "genre-column"
    return genres

movies["genres"]=movies["genres"].apply(genre_extractor)
movies["genres"]

0       [action, adventure, fantasy, science-fiction]
1                        [adventure, fantasy, action]
2                          [action, adventure, crime]
3                    [action, crime, drama, thriller]
4                [action, adventure, science-fiction]
                            ...                      
4804                        [action, crime, thriller]
4805                                [comedy, romance]
4806               [comedy, drama, romance, tv-movie]
4807                                               []
4808                                    [documentary]
Name: genres, Length: 4809, dtype: object

In [12]:
# 'keywords' column
movies['keywords'].head()
movies['keywords'][0]

# to make all words lower-case
movies["keywords"]=movies["keywords"].str.lower()

# Here we only need "names" from "key-words"
# we also need to remove " " As 'Science','Fiction' & 'Science Fiction' can be represent as different  
def keywords_extractor(df):
    keywords=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return keywords

movies["keywords"]=movies["keywords"].apply(keywords_extractor)
movies["keywords"]

0       [culture-clash, future, space-war, space-colon...
1       [ocean, drug-abuse, exotic-island, east-india-...
2       [spy, based-on-novel, secret-agent, sequel, mi...
3       [dc-comics, crime-fighter, terrorist, secret-i...
4       [based-on-novel, mars, medallion, space-travel...
                              ...                        
4804    [united-states–mexico-barrier, legs, arms, pap...
4805                                                   []
4806    [date, love-at-first-sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream-girl]
Name: keywords, Length: 4809, dtype: object

In [13]:
# 'overview'
movies["overview"].head()
movies["overview"][0]


# make all words lower-case
movies["overview"]=movies["overview"].str.lower()

# Remove "punctuation" 
movies["overview"]=movies['overview'].str.replace('[^\w\s]','')

# Remove "stop-words" as those are not important 
# Also need to "Stem" these words , As "play" , "plays" & "playing" can be represent as different.
stop_words=stopwords.words('english')
ps=PorterStemmer()

def preprocess(words):
    preprocessed=[word for word in words.split() if word not in stop_words]  # removoing stop-words
    preprocessed=[ps.stem(word) for word in preprocessed]                   # steming 
    preprocessed=' '.join(preprocessed) # join words to sentence
    return preprocessed

movies['overview']=movies['overview'].apply(preprocess)
movies['overview']

C:\Users\jitan\anaconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


0       22nd centuri parapleg marin dispatch moon pand...
1       captain barbossa long believ dead come back li...
2       cryptic messag bond past send trail uncov sini...
3       follow death district attorney harvey dent bat...
4       john carter warweari former militari captain w...
                              ...                        
4804    el mariachi want play guitar carri famili trad...
4805    newlyw coupl honeymoon upend arriv respect sister
4806    sign seal deliv introduc dedic quartet civil s...
4807    ambiti new york attorney sam sent shanghai ass...
4808    ever sinc second grade first saw et extraterre...
Name: overview, Length: 4809, dtype: object

In [14]:
#'cast' column
movies['cast'].head()
movies['cast'][0]

# to make all words lower-case
movies['cast']=movies['cast'].str.lower()

# Here we only need "names" from "key-words"
# As there are many "cast" in a movie , I'm just taking "5 front casts"
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def casts_extractor(df):
    cast=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return cast[:5]

movies['cast']=movies['cast'].apply(casts_extractor)
movies['cast'][1]

['johnny-depp',
 'orlando-bloom',
 'keira-knightley',
 'stellan-skarsgård',
 'chow-yun-fat']

In [15]:
#'crew' column
movies['crew'].head()
movies['crew'][0]

# to make all words lower-case
movies['crew']=movies['crew'].str.lower()

# Crews are important but it's lots of information to process & also not all crew members equally important for our task 
# So I'm taking only "director" & "producer"
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def director_extractor(df):
    """ Extract Director name from Crew Members"""
    name=[i["name"].replace(" ","-") for i in eval(df) if i["job"]=="director"]    # extract only list of "director-names" 
    return [name[0] if len(name)!=0 else ""]  # if there are multiple "directors" return 1st && if no-director name listed then return "" 

movies['director']=movies['crew'].apply(director_extractor)

def producer_extractor(df):
    """ Extract Producer name from Crew Members"""
    name=[i["name"].replace(" ","-") for i in eval(df) if i["job"]=="producer"]    # extract only list of "producer-names" 
    return [name[0] if len(name)!=0 else ""]  # if there are multiple "producers" return 1st && if no-producer name listed then return "" 

movies['producer']=movies['crew'].apply(producer_extractor)


## As we extracted valuable info. from "crew" column, so we can now drop that
movies.drop(['crew'],axis=1,inplace=True)

## In Some movies there are same "producer" as "director". So we have take one when they are same 
movies[['producer','director']]


In [16]:
#'production_companies' column
movies['production_companies'].head()
movies['production_companies'][0]

# to make all words lower-case
movies['production_companies']=movies['production_companies'].str.lower()

# I did not know "production_companies" important or not but for me, some times I ignore movies thats are not belongs to known "production_companies"
# so I'm keeping this column you can skip it
# Here I'm taking only 2 front-line "production_companies" 
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def production_companies_extractor(df):
    """ Extract production_companies name """
    name=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return name[:2] 

movies['production_companies']=movies['production_companies'].apply(production_companies_extractor)
movies['production_companies']

0       [ingenious-film-partners, twentieth-century-fo...
1         [walt-disney-pictures, jerry-bruckheimer-films]
2                             [columbia-pictures, danjaq]
3                      [legendary-pictures, warner-bros.]
4                                  [walt-disney-pictures]
                              ...                        
4804                                  [columbia-pictures]
4805                                                   []
4806    [front-street-pictures, muse-entertainment-ent...
4807                                                   []
4808         [rusty-bear-entertainment, lucky-crow-films]
Name: production_companies, Length: 4809, dtype: object

# Prepare data-set for Recommender-System

In [17]:
## We need to merge ['genres','keywords','overview','production_companies','cast','director','producer']

# we need to convert 'tags' column values from list to string & after that we can add it with "overview" column
# And we also need them as string to do modelling
movies["tags"]=movies['genres']+movies['keywords']+movies['cast']+movies['director']+movies['producer']+movies['production_companies']
movies['tags']=movies['tags'].apply(' '.join)   # converts list to strings

# Now we can add "overview" column to "tag"
movies['tags']=movies['tags']+movies['overview']

movies['tags']


0       action adventure fantasy science-fiction cultu...
1       adventure fantasy action ocean drug-abuse exot...
2       action adventure crime spy based-on-novel secr...
3       action crime drama thriller dc-comics crime-fi...
4       action adventure science-fiction based-on-nove...
                              ...                        
4804    action crime thriller united-states–mexico-bar...
4805    comedy romance edward-burns kerry-bishé marsha...
4806    comedy drama romance tv-movie date love-at-fir...
4807    daniel-henney eliza-coupe bill-paxton alan-ruc...
4808    documentary obsession camcorder crush dream-gi...
Name: tags, Length: 4809, dtype: object

In [18]:
## For our System we now need 3 columns "movie_id","title","tag"
movie_refined_data=movies[["movie_id","title","tags"]]
movie_refined_data.head()
movie_refined_data.shape  #4809,3

(4809, 3)

In [19]:
cv=CountVectorizer(max_features=5000)
word_vectors=cv.fit_transform(movie_refined_data["tags"]).toarray()
#cv.get_feature_names()

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
word_vector=tfidf.fit_transform(movie_refined_data["tags"]).toarray()

word=tfidf.get_feature_names()
#word

C:\Users\jitan\anaconda3\envs\yolov4-gpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Calculating Similarity or distance-metrics
* Here we need to calculate similarity of each movie with rest of movies
* In High-Dimensional Space "Eucleadian Distance" is not a good measure so we are using "Cosine Similarity"
* 

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix=cosine_similarity(word_vector)
# creates a matrix containing matrix of "similarity_score" of each movies with each other
similarity_matrix.shape

(4809, 4809)

In [30]:
type(similarity_matrix)

numpy.ndarray

In [22]:
## Recommendation Engine
def recommend(movie_name):
    #movie_name=movie_name.title()
    if movie_name in movie_refined_data["title"].values: # checks if movie name in data or not
        movie_index=movie_refined_data[movie_refined_data["title"]==(movie_name)].index[0] # return movie_Id from movie name
        similarity=similarity_matrix[movie_index]
        recommend_movies=sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[1:6] # return top-5 most similar(movie_Id,similarity_matrix)

        print("Your Recommended Movies:-")
        for movie in recommend_movies:
            print(movie_refined_data.iloc[movie[0]].title)
    else:
        print("Sorry!!! Try with another Movie name")

    #return recommend_movie_index

    


In [23]:
## Predict Your Next Movie
recommend("Pirates of the Caribbean: On Stranger Tides")


Your Recommended Movies:-
Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: At World's End
Kangaroo Jack
Prince of Persia: The Sands of Time


## Create Data for Streamlit app

In [25]:
data=movie_refined_data[["movie_id","title"]]
#data=data.set_index("movie_id")
#data.to_csv("movie_data.csv")   # write data to csv file

np.save('similarity_matrix.npy',similarity_matrix)  # write similarity matrix to npy file

In [ ]:
## For Movie recommendation we just need "data" & "similarity_matrix"